In [101]:
import pandas as pd

In [102]:
data_df = pd.read_csv("DataSample.csv")
data_df.rename(columns=lambda x: x.strip(), inplace=True)

## Part 1

data_df.drop_duplicates(["TimeSt", "Latitude", "Longitude"], keep="last", inplace=True)

data_df
# data.count
# data.count_distinct

_ID                   TimeSt Country Province              City  \
0      4516516  2017-06-21 00:00:00.143      CA       ON          Waterloo   
1      4516547  2017-06-21 18:00:00.193      CA       ON            London   
2      4516550  2017-06-21 15:00:00.287      CA       ON            Guelph   
3      4516600  2017-06-21 15:00:00.307      CA       ON         Stratford   
4      4516613  2017-06-21 15:00:00.497      CA       ON         Stratford   
5      4516693  2017-06-21 14:00:00.597      CA       ON         Kitchener   
6      4516771  2017-06-21 10:00:00.873      CA       ON            Sarnia   
8      4516915  2017-06-21 15:00:01.310      CA       ON            London   
9      4516953  2017-06-21 16:00:01.700      CA       ON         Kitchener   
10     4516966  2017-06-21 01:00:01.787      CA       ON         Kitchener   
11     4517044  2017-06-21 21:00:00.327      CA       ON           Windsor   
12     4517047  2017-06-21 12:00:00.950      CA       ON            London   
13     4517081  2017-06-21 22:00:00.753      CA       ON           Windsor   
14     4517175  2017-06-21 19:00:01.087      CA       ON      Saint Thomas   
15     4517220  2017-06-21 18:00:01.213      CA       ON          Waterloo   
16     4517250  2017-06-21 03:00:01.393      CA       ON           Chatham   
18     4517326  2017-06-21 04:00:00.367      CA       ON            Guelph   
19     4517347  2017-06-21 03:00:00.287      CA       ON           Windsor   
20     4517349  2017-06-21 14:00:00.420      CA       ON           Chatham   
21     4517420  2017-06-21 08:00:00.427      CA       ON            Guelph   
22     4517466  2017-06-21 03:00:01.157      CA       ON          Waterloo   
23     4517518  2017-06-21 21:00:01.333      CA       ON          Waterloo   
24     4517538  2017-06-21 21:00:01.427      CA       ON         Kitchener   
25     4517607  2017-06-21 01:00:01.623      CA       ON           Chatham   
26     4517697  2017-06-21 12:00:01.790      CA       ON           Windsor   
27     4517720  2017-06-21 22:00:01.843      CA       ON         Kitchener   
28     4517809  2017-06-21 17:00:00.030      CA       ON            London   
29     4517811  2017-06-21 12:00:00.150      CA       ON         Cambridge   
30     4517905  2017-06-21 17:00:00.183      CA       ON           Windsor   
31     4517999  2017-06-21 12:00:00.643      CA       ON         Kitchener   
...        ...                      ...     ...      ...               ...   
21992  5613570  2017-06-21 12:22:55.203      CA       ON           Toronto   
21993  5613594  2017-06-21 00:22:56.987      CA       AB          Edmonton   
21994  5613616  2017-06-21 03:22:56.990      CA       ON           Toronto   
21995  5613624  2017-06-21 10:22:52.793      CA       ON       Scarborough   
21996  5613651  2017-06-21 12:22:53.030      CA       ON        Belleville   
21997  5613662  2017-06-21 22:22:53.100      CA       ON           Toronto   
21998  5613677  2017-06-21 19:22:53.130      CA       ON           Toronto   
21999  5613763  2017-06-21 08:22:53.193      CA       ON        Burlington   
22000  5613790  2017-06-21 06:22:54.370      CA       ON        Burlington   
22001  5613826  2017-06-21 04:22:53.000      CA       ON       Mississauga   
22002  5613874  2017-06-21 13:22:53.103      CA       ON            Ottawa   
22003  5613968  2017-06-21 17:22:53.147      CA       ON       Scarborough   
22004  5614047  2017-06-21 01:22:53.280      CA       ON       Mississauga   
22005  5614136  2017-06-21 17:22:53.340      CA       ON            Aurora   
22007  5614181  2017-06-21 07:22:53.863      CA       AB          Red Deer   
22009  5614303  2017-06-21 22:22:55.020      CA       ON           Toronto   
22010  5614377  2017-06-21 22:22:53.500      CA       ON            Barrie   
22011  5614409  2017-06-21 20:22:53.367      CA       ON       Mississauga   
22012  5614446  2017-06-21 07:22:53.747      CA       AB           Calgary   
22013  5614486  2017-06-21 04:22:

In [103]:
## Part 2

poi_df = pd.read_csv("POIList.csv")
poi_df.rename(columns=lambda x: x.strip(), inplace=True)
poi_map = {}

for _, row in poi_df.iterrows():
    poi_map[(row["Latitude"], row["Longitude"])] = row["POIID"]
    
def calc_dist(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5
    
def closest_poi(row, mapping):
    distances = {k: calc_dist([row["Latitude"], row["Longitude"]], k) for k in mapping.keys()}
    place = min(distances, key=distances.get)
    return mapping[place]

data_df['ClosestPOI'] = data_df.apply(lambda row: closest_poi(row, poi_map), axis=1)

data_df.groupby(['ClosestPOI']).agg(['count'])

_ID TimeSt Country Province  City Latitude Longitude
           count  count   count    count count    count     count
ClosestPOI                                                       
POI2        9698   9698    9698     9698  9698     9698      9698
POI3        9817   9817    9817     9817  9817     9817      9817
POI4         484    484     484      484   484      484       484

In [111]:
## Part 3

def poi_dist(row, mapping):
    return min([calc_dist([row["Latitude"], row["Longitude"]], k) for k in mapping.keys()])

data_df['ClosestDist'] = data_df.apply(lambda row: poi_dist(row, poi_map), axis=1)

In [123]:
# 1)

summary = data_df[['ClosestPOI', 'ClosestDist']].groupby(['ClosestPOI']).agg(['mean', 'std'])

# Since POI1 and POI2 are the same geographical location
poi_1 = summary.loc["POI2"]
poi_1.rename("POI1", inplace=True)

summary = summary.append(poi_1)
summary

ClosestDist          
                  mean       std
ClosestPOI                      
POI2          3.348183   3.85849
POI3          5.537951   2.85869
POI4          8.810411  28.67513
POI1          3.348183   3.85849

In [163]:
# 2)

poi_range = data_df[['ClosestPOI', 'ClosestDist']].groupby(['ClosestPOI']).agg(['max', 'count'])
poi_range.columns = poi_range.columns.droplevel()
poi_range['density'] = poi_range.apply(lambda row: row['count']/row['max'], axis=1)
poi_range.drop(['count'], axis=1, inplace=True)
poi_range.rename(columns={'max': 'radius'}, inplace=True)

poi_1_range = poi_range.loc["POI2"]
poi_1_range.rename("POI1", inplace=True)

poi_range = poi_range.append(poi_1_range)
poi_range

radius     density
ClosestPOI                        
POI2         24.851937  390.231148
POI3         20.155378  487.066038
POI4        192.704991    2.511611
POI1         24.851937  390.231148

_ID TimeSt Country Province  City Latitude Longitude ClosestDist
           count  count   count    count count    count     count       count
ClosestPOI                                                                   
POI2        9698   9698    9698     9698  9698     9698      9698        9698
POI3        9817   9817    9817     9817  9817     9817      9817        9817
POI4         484    484     484      484   484      484       484         484